In [ ]:
from pyspark.sql.functions import col

print("Aplicando transformações e corrigindo tipos...")

# A lógica simplificada: Converte direto para Float.
# O que for "NaN" ou vazio vira NULL, e o .fillna(0.0) transforma NULL em 0.
df_silver = df_raw.select(
    col("NU_INSCRICAO").alias("id_inscricao"),
    col("NU_ANO").alias("ano"),
    col("TP_SEXO").alias("sexo"),
    col("NO_MUNICIPIO_PROVA").alias("cidade_prova"),
    col("SG_UF_PROVA").alias("uf_prova"),
    
    # Notas: Cast direto (Se falhar, vira null, depois vira 0)
    col("NU_NOTA_CN").cast("float").alias("nota_ciencias_natureza"),
    col("NU_NOTA_CH").cast("float").alias("nota_humanas"),
    col("NU_NOTA_LC").cast("float").alias("nota_linguagens"),
    col("NU_NOTA_MT").cast("float").alias("nota_matematica"),
    col("NU_NOTA_REDACAO").cast("float").alias("nota_redacao")
).fillna(0.0)

print("Dados transformados! Visualizando amostra:")
df_silver.show(5)

In [ ]:
# Configurações do Banco
db_url = "jdbc:postgresql://localhost:5432/dados_inep"
usuario = "admin"
senha = "l1l2r1r2"
tabela_destino = "public.silver_enem"

print(f"Gravando dados na tabela {tabela_destino}...")

try:
    df_silver.write \
        .format("jdbc") \
        .option("url", db_url) \
        .option("dbtable", tabela_destino) \
        .option("user", usuario) \
        .option("password", senha) \
        .option("driver", "org.postgresql.Driver") \
        .mode("overwrite") \
        .save()
    
    print("✅ GRAVAÇÃO CONCLUÍDA COM SUCESSO!")

except Exception as e:
    print("❌ ERRO AO GRAVAR NO BANCO:")
    print(e)